In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
from collections import Counter
import spacy
import en_core_web_sm
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer
from xgboost import XGBClassifier
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
import re
from nltk.stem import PorterStemmer
nlp = en_core_web_sm.load()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nlp = en_core_web_sm.load()
en_stop = set(stopwords.words('english'))


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Carregando datasets

train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

sample_submission = pd.read_csv('../input/quora-insincere-questions-classification/sample_submission.csv')

In [ ]:
#Tornar o dataset de treino balanceado.
X_train = train[train['target'] == 1]

X_train = X_train.append(train[train['target'] == 0]
                         .sample(n = len(X_train))).reset_index(drop = True)

print(X_train.target.value_counts())

In [ ]:
test.head()

In [ ]:
#Pré processamento dos dados
def pre(texto):
    texto = texto.lower()
    texto = re.sub('\[.*?\]', '', texto)
    texto = re.sub('https?://\S+|www\.\S+', '', texto)
    texto = re.sub('<.*?>+', '', texto)
    texto = re.sub('[%s]' % re.escape(string.punctuation), '', texto)
    texto = re.sub('\n', '', texto)
    texto = re.sub('\w*\d\w*', '', texto)
    return texto

In [ ]:
#Recuperando classes das instâncias
y_target_train = X_train.target.values


#Pré Processamento
X_train['clean_text'] = X_train.question_text.apply(str).apply(lambda text: pre(text))
test['clean_text'] = test.question_text.apply(str).apply(lambda text: pre(text))


#Obtendo os valores de treino e teste das instâncias
X = X_train.clean_text.values
y_test = test.clean_text.values

In [ ]:
#Campo Clean_Text depois do pré processamento
X_train.head()

In [ ]:
#Extração das features
vectorizer = TfidfVectorizer(use_idf=True, stop_words='english')

tfidf_model = vectorizer.fit(X)

tfidf_train = tfidf_model.transform(X)
tfidf_test = tfidf_model.transform(y_test)

In [ ]:
#Modelo que será avaliado é o NaiveBayes
models = [
          ('MNB', MultinomialNB())
        ]

for name, model in models:
        model = OneVsRestClassifier(model)
        clf = model.fit(tfidf_train, y_target_train)
        preds = clf.predict(tfidf_test)

In [ ]:
#Criando um vetor unidimensional com os qids e predições.
preds = pd.Series(preds)

#Salvando os dados finais.
sample_submission['prediction'] = preds
print(sample_submission)
sample_submission.to_csv('submission.csv', index=False)